# TP -- Pracical work

### Table of Contents

* [Introduction](#introduction)
* [2. Economic analysis, computing LCOE, system LCOE](#2.LCOE)
    * [Q2.1. LCOE of renewable + Flex system](#2.1.Question)
* [3. Optimisation of operation](#3.Operation)
    * [Q3.1. Optimisation results and Lagrange multipliers](#3.1.Question)
    * [Q3.2. Storage optimisation](#3.2.Question)
* [4. Optimisation of planning](#4.Planning)
    * [Q4.1. Optimisation results and Lagrange multipliers](#4.1.Question)



## Introduction <a class="anchor" id="introduction"></a>

Question asked here can generally be answered by copy-past of code given in folder "BasicFunctionalities" + small modifications/additions of yoru own.


In [11]:
import os
if os.path.basename(os.getcwd())=="BasicFunctionalities":
    os.chdir('..') ## to work at project root  like in any IDE
if os.path.basename(os.getcwd())=="Reponses_TP":
    os.chdir('../..') ## to work at project root  like in any IDE

InputFolder='Data/input/'

#region importation of modules
import numpy as np
import pandas as pd
import seaborn as sns
import csv
import datetime
import copy
import qgrid
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from sklearn import linear_model
from functions.f_operationModels import *
from functions.f_optimization import *
from functions.f_graphicalTools import *
import sys

#endregion

#region Solver and data location definition

InputFolder='Data/input/'

if sys.platform != 'win32':
    myhost = os.uname()[1]
else : myhost = ""
if (myhost=="jupyter-sop"):
    ## for https://jupyter-sop.mines-paristech.fr/ users, you need to
    #  (1) run the following to loanch the license server
    if (os.system("/opt/mosek/9.2/tools/platform/linux64x86/bin/lmgrd -c /opt/mosek/9.2/tools/platform/linux64x86/bin/mosek.lic -l lmgrd.log")==0):
        os.system("/opt/mosek/9.2/tools/platform/linux64x86/bin/lmutil lmstat -c 27007@127.0.0.1 -a")
    #  (2) definition of license
    os.environ["MOSEKLM_LICENSE_FILE"] = '@jupyter-sop'

BaseSolverPath='/Users/robin.girard/Documents/Code/Packages/solvers/ampl_macosx64' ### change this to the folder with knitro ampl ...
## in order to obtain more solver see see https://ampl.com/products/solvers/open-source/
## for eduction this site provides also several professional solvers, that are more efficient than e.g. cbc
sys.path.append(BaseSolverPath)
solvers= ['gurobi','knitro','cbc'] # try 'glpk', 'cplex'
solverpath= {}
for solver in solvers : solverpath[solver]=BaseSolverPath+'/'+solver
solver= 'mosek' ## no need for solverpath with mosek.
#endregion

#region - general economic assumption
General = pd.read_csv(InputFolder+'GeneralEconomicAssumptions.csv',sep=',',decimal='.',skiprows=0,comment="#")
col_def = {}#  { 'A': { 'editable': True } }
qgrid_widget = qgrid.show_grid(General, show_toolbar=True, column_definitions=col_def)
qgrid_widget
#endregion

#region - general production assumption
Production = pd.read_csv(InputFolder+'ProductionEconomicAssumptions.csv',sep=',',decimal='.',skiprows=0,comment="#")
Production=Production[Production["AREAS"]=="FR"]
qgrid_widget = qgrid.show_grid(Production, show_toolbar=True, column_definitions=col_def)
qgrid_widget

#endregion


QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…


## 2. Economic analysis, computing LCOE, system LCOE <a class="anchor" id="2.LCOE"></a>

### Q2.1. LCOE of renewable + Flex system <a class="anchor" id="2.1.Question"></a>
Read and understand jupyter notebook optim-Operation.ipynb.
Try to imagine systems that are feasible and compute the corresponding system LCOE in the two following cases:
- Nuke + CCG + hydro
- Renewable + CCG + hydro
You will need to set the installed power and produced energy for each of these production mean. Try to get inspiration from preceding section.
There are no "right answer", the idea here is to have rough estimates that will be improved later.
Try adding significant amount of electrolyser/fuel cells to replace CCG. How much capacity would you install and see how it changes the results ?
Try changing the thermal sensitivity of consumption.
You can also analyse the effect of discount rate, or carbon tax, or cost of gaz (by assumign e.g. it is a very expensive biogaz).

In [12]:
General = pd.read_csv(InputFolder+'GeneralEconomicAssumptions.csv',sep=',',decimal='.',skiprows=0,comment="#")
col_def = {}#  { 'A': { 'editable': True } }
qgrid_widget = qgrid.show_grid(General, show_toolbar=True, column_definitions=col_def)
qgrid_widget

""""General=qgrid_widget.get_changed_df()
General"""

ProductionTech0 = pd.read_csv(InputFolder+'ProductionTechnicalAssumptions.csv',sep=',',decimal='.',skiprows=0,comment="#")
ProductionTech0=ProductionTech0[ProductionTech0["AREAS"]=="FR"]
qgrid_widget = qgrid.show_grid(ProductionTech0, show_toolbar=True, column_definitions=col_def)
qgrid_widget
ProductionTech0=qgrid_widget.get_changed_df()
ProductionTechPlus0=ProductionTech0.merge(Production, how='inner', left_on=["AREAS","TECHNOLOGIES"], right_on=["AREAS","TECHNOLOGIES"])



Tech=['OldNuke','CCG','HydroReservoir','HydroRiver']
ProductionTechPlus0=ProductionTechPlus0[ProductionTechPlus0['TECHNOLOGIES'].isin(Tech)]

ProductionTechPlus0.loc[3,'CAPEX']=3000


r=(General.discountPercent[0]/100)
df0 = ProductionTechPlus0
df0["LLr"] = round((1+r)/r*(1-(1+r)**(-df0['LL'])),2)
df0["LCOE"] = (df0.CAPEX/df0.LLr + df0.dismantling/((1+r)**(df0.LL)) + df0.FOM) * (df0.capacity/(1000*df0.ProducedEnergy)) + df0.Variable
df0.drop(['RampConstraintPlus','RampConstraintMoins','RampConstraintPlus2','RampConstraintMoins2','YEAR_y'],axis='columns',inplace=True)
df0["TotalCost"]=df0.LCOE*df0.ProducedEnergy



In [13]:
import os
if os.path.basename(os.getcwd())=="BasicFunctionalities":
    os.chdir('..') ## to work at project root  like in any IDE
InputFolder='Data/input/'
import pandas as pd
import qgrid 
import numpy as np

General = pd.read_csv(InputFolder+'GeneralEconomicAssumptions.csv',sep=',',decimal='.',skiprows=0,comment="#")
col_def = {}#  { 'A': { 'editable': True } }
qgrid_widget = qgrid.show_grid(General, show_toolbar=True, column_definitions=col_def)
qgrid_widget

""""General=qgrid_widget.get_changed_df()
General"""

ProductionTech = pd.read_csv(InputFolder+'ProductionTechnicalAssumptions.csv',sep=',',decimal='.',skiprows=0,comment="#")
ProductionTech=ProductionTech[ProductionTech["AREAS"]=="FR"]
qgrid_widget = qgrid.show_grid(ProductionTech, show_toolbar=True, column_definitions=col_def)
qgrid_widget
ProductionTech=qgrid_widget.get_changed_df()
ProductionTechPlus=ProductionTech.merge(Production, how='inner', left_on=["AREAS","TECHNOLOGIES"], right_on=["AREAS","TECHNOLOGIES"])



Tech=['Solar','CCG','HydroReservoir','HydroRiver','WindOnShore']
df=ProductionTechPlus[ProductionTechPlus['TECHNOLOGIES'].isin(Tech)]

ProductionTechPlus.loc[3,'CAPEX']=3000

r=(General.discountPercent[0]/100)
df = ProductionTechPlus
df["LLr"] = round((1+r)/r*(1-(1+r)**(-df['LL'])),2)
df["LCOE"] = (df.CAPEX/df.LLr + df.dismantling/((1+r)**(df.LL)) + df.FOM) * (df.capacity/(1000*df.ProducedEnergy)) + df.Variable
df.drop(['RampConstraintPlus','RampConstraintMoins','RampConstraintPlus2','RampConstraintMoins2','YEAR_y'],axis='columns',inplace=True)



ProductionTech2 = pd.read_csv(InputFolder+'ProductionTechnicalAssumptions.csv',sep=',',decimal='.',skiprows=0,comment="#")
ProductionTech2=ProductionTech2[ProductionTech2["AREAS"]=="FR"]
qgrid_widget = qgrid.show_grid(ProductionTech, show_toolbar=True, column_definitions=col_def)
qgrid_widget
ProductionTech2=qgrid_widget.get_changed_df()
df2=ProductionTech2.merge(Production, how='inner', left_on=["AREAS","TECHNOLOGIES"], right_on=["AREAS","TECHNOLOGIES"])

Tech=['Solar','CCG','HydroReservoir','HydroRiver','WindOnShore']
df2=df2[df2['TECHNOLOGIES'].isin(Tech)]

df2.drop(['RampConstraintPlus','RampConstraintMoins','RampConstraintPlus2','RampConstraintMoins2','YEAR_y'],axis='columns',inplace=True)
df2.loc[2,'capacity']=30000
df2.loc[6,'capacity']=160000
df2.loc[8,'capacity']=12000
df2.loc[9,'capacity']=12000
df2.loc[10,'capacity']=80000


r=(General.discountPercent[0]/100)
df2["LoadFactor"] = 1000*df.ProducedEnergy/(8.76*df.capacity)
df2["ProducedEnergy"]=(df2["capacity"]*df2["LoadFactor"]*8.76)/1000
df2["LLr"] = round((1+r)/r*(1-(1+r)**(-df2['LL'])),2)
df2["LCOE"]=(df2.CAPEX/df2.LLr + df2.dismantling/((1+r)**(df2.LL)) + df2.FOM) * (df2.capacity/(1000*df2.ProducedEnergy)) + df2.Variable
df2["TotalCost"]=df2["ProducedEnergy"]*df2["LCOE"]



df2

,YEAR_x,AREAS,TECHNOLOGIES,capacity,ProducedEnergy,EnergyNbhourCap,CAPEX,dismantling,Variable,FOM,LL,CL,CO2Emission,LoadFactor,LLr,LCOE,TotalCost
2,2019,FR,CCG,30000,96.500000,0,900,100,50,36,30,2,470,0.367199,17.98,86.338078,8331.624524
6,2019,FR,WindOnShore,160000,311.409396,0,1300,50,0,45,30,2,0,0.222181,17.98,68.189846,21234.958688
8,2019,FR,HydroReservoir,12000,56.571429,2100,1500,10,0,20,80,10,0,0.538160,24.87,17.128252,968.969690
9,2019,FR,HydroRiver,12000,36.000000,0,1500,10,0,20,80,10,0,0.342466,24.87,26.915825,968.969690
10,2019,FR,Solar,80000,91.666667,0,800,30,0,20,25,1,0,0.130803,16.25,70.240820,6438.741864


In [14]:
###Calcul Moyenne pondérée LCOE
df2_c=df2
df0_c=df0

df2_c["LCOExEnergieProduite"]=df2_c.ProducedEnergy * df2_c.LCOE
df0_c["LCOExEnergieProduite"]=df0_c.ProducedEnergy * df0_c.LCOE

LCOE_moy_pond1=int((df0_c.LCOExEnergieProduite.sum())/(df0_c.ProducedEnergy.sum()))
LCOE_moy_pond2=int((df2_c.LCOExEnergieProduite.sum())/(df2_c.ProducedEnergy.sum()))

Cout_energie_Scenario1=int(df0.TotalCost.sum())
Cout_energie_Scenario2=int(df2.TotalCost.sum())
Energy_prod1=int(df0.ProducedEnergy.sum())
Energy_prod2=int(df2.ProducedEnergy.sum())
l=[LCOE_moy_pond1,Cout_energie_Scenario1,Energy_prod1]
l2=[LCOE_moy_pond2,Cout_energie_Scenario2,Energy_prod2]

resl=[l,l2]
resl

[[60, 31067, 514], [64, 37943, 592]]

In [15]:

res = pd.DataFrame(resl, index = ['Scenario Nuke','Scenario Renewables'], columns=['LCOE moyen Pondéré','Cout_tot','Energie produite'])
print("Cout scenario1:  " + str(Cout_energie_Scenario1) ,"Cout scenario2:  " + str(Cout_energie_Scenario2) )
res

Cout scenario1:  31067 Cout scenario2:  37943


,LCOE moyen Pondéré,Cout_tot,Energie produite
Scenario Nuke,60,31067,514
Scenario Renewables,64,37943,592


## 3. Optimisation of operation <a class="anchor" id="3.Operation"></a>

### Q3.1. Optimisation results and Lagrange multipliers <a class="anchor" id="3.1.Question"></a>
Read and understand jupyter notebook optim-Operation.ipynb. Can you explain the value of all lagrange multiplier in a simple case with two production means ?
With one area and then with two. How is it possible to generate negative lagrange multipliers ? Propose a simple case where they appear and explain the corresponding values.


In [16]:
# Put your answer here

### Q3.2. Storage optimisation <a class="anchor" id="3.2.Question"></a>
Read and understand jupyter notebook optim-storage.ipynb. What is the maximal price for the storage system to be profitable today ?
Apply an homothetic transformation to increase the variance of the prices but without changing the mean (and by keeping positive prices).
How does it change the preceding results ?

In [17]:
# Put your answer here


## 4. Optimisation of planning <a class="anchor" id="4.Planning"></a>

### Q4.1. Optimisation results and Lagrange multipliers <a class="anchor" id="4.1.Question"></a>

**a - System LCOE**

Read and understand jupyter notebook optim-storage.ipynb. Try using it to adapt your answer to Q2.1.

In [18]:
# Put your answer here

**a - Loss of load expectation**

Today the expected loss of load is of 3 hours in average per year, try to find how to set the variable cost of "curtailment"
(consumption that is cut, the cost is initially set to 3000 €/MWh) to have the value of this expected number at 3.1.
Try to find in the CRE or RTE or ENEDIS website the "official value" in France.

In [19]:
# Put your answer here